In [1]:

import sys, os, git
import shutil
import numpy as np
import subprocess

from utilities import detect_saving_folder, try_to_copy_file, \
                      return_file_content, detect_block_code, \
                      detect_block_types, create_file, detect_last_matching_line, \
                      append_content, detect_method_boundaries, detect_existing_lines, \
                      detect_methods

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")

path_to_docs = git_path + "/docs/source/chapters/"

# make sure the documentaiton was found
assert os.path.exists(path_to_docs), """Documentation files not found"""

if os.path.exists("generated-codes/") is False:
    os.mkdir("generated-codes/")

mycwd = os.getcwd()
for chapter_id in [1, 2, 3, 4]: # np.arange(10):
    filename = path_to_docs + "chapter"+str(chapter_id)+".rst"
    created_files, created_tests = [], []
    if os.path.exists(filename):
        folder = detect_saving_folder(chapter_id)
        created_files = try_to_copy_file(chapter_id, created_files)
        file_content = return_file_content(filename)
        block_contents, block_names = detect_block_code(file_content)
        block_types = detect_block_types(block_contents)
        created_files, created_tests = create_file(block_contents, block_names,
                                                   created_files, created_tests, folder)
        for content, name, type in zip(block_contents, block_names, block_types):
            append_content(folder, name, content, type)
        # Run the tests
        os.chdir(folder)
        for test_file in created_tests:
            print("TEST --", "chapter"+str(chapter_id)+".rst", "--", test_file)
            subprocess.call(["python3", test_file])
        os.chdir(mycwd)

TEST -- chapter1.rst -- test_First.py
TEST -- chapter2.rst -- test_Prepare.py
TEST -- chapter3.rst -- test_InitializeSimulation.py
TEST -- chapter4.rst -- test_MinimizeEnergy.py


In [2]:
chapter_id = 5
filename = path_to_docs + "chapter"+str(chapter_id)+".rst"
created_files, created_tests = [], []
if os.path.exists(filename):
    folder = detect_saving_folder(chapter_id)
    created_files = try_to_copy_file(chapter_id, created_files)
    file_content = return_file_content(filename)
    block_contents, block_names = detect_block_code(file_content)
    block_types = detect_block_types(block_contents)
    created_files, created_tests = create_file(block_contents, block_names,
                                                created_files, created_tests, folder)
    for content, name, type in zip(block_contents, block_names, block_types):
        append_content(folder, name, content, type)

In [10]:
original_file_content = return_file_content(folder+name+".py")
existing_methods = detect_methods(original_file_content)
new_method = detect_methods(content)
original_start, original_end = detect_method_boundaries(new_method[0], original_file_content)
new_start, new_end = detect_method_boundaries(new_method[0], content)
location_lines= detect_existing_lines(content[new_start:new_end],
                                    original_file_content[original_start:original_end])

In [22]:
method_name = new_method[0]

In [23]:
original_start_init = None
original_end_init = []
for cpt, l in enumerate(original_file_content):
    if ("def" in l) & (method_name in l):
        original_start_init = cpt
    elif (":" in l) & ("def" in l) & (method_name not in l):
        original_end_init.append(cpt)      
if len(original_end_init) > 0:  
    original_end_init = original_end_init[0]
else:
    original_end_init = cpt
if original_end_init < original_start_init:
    original_end_init = cpt

In [25]:
original_end_init

27

In [ ]:

    for content, name, type in zip(block_contents, block_names, block_types):
        append_content(folder, name, content, type)
    # Run the tests
    os.chdir(folder)
    for test_file in created_tests:
        print("TEST --", "chapter"+str(chapter_id)+".rst", "--", test_file)
        subprocess.call(["python3", test_file])
    os.chdir(mycwd)